# Data loading, storage, and file formats

In [53]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [54]:
%pwd

'/home/lex/BROSCORP/Pycharm/Projects/DataAnalysis/src'

## Reading and Writing Data in Text Format

In [55]:
!cat data/io/io_1.csv
df = pd.read_csv('data/io/io_1.csv')
df

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [56]:
pd.read_table('data/io/io_1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [57]:
!cat data/io/io_2.csv
pd.read_csv('data/io/io_2.csv', header=None)
pd.read_csv('data/io/io_2.csv', names=['a', 'b', 'c', 'd', 'message'])

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [58]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data/io/io_2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [59]:
!cat data/io/ierch_index.csv
parsed = pd.read_csv('data/io/ierch_index.csv', index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [60]:
list(open('data/io/reg_exp.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491']

In [61]:
# Reg exp like a separator for read
result = pd.read_table('data/io/reg_exp.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [62]:
!cat data/io/row_skip.csv
pd.read_csv('data/io/row_skip.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [63]:
!cat data/io/mis_val.csv
result = pd.read_csv('data/io/mis_val.csv')
result
pd.isnull(result)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [64]:
# custom markers of missing values
result = pd.read_csv('data/io/mis_val.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [65]:
# custom markers of missing values with separate logic
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('data/io/mis_val.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading text files in pieces

In [66]:
result = pd.read_csv('data/io/big.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [67]:
pd.read_csv('data/io/big.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [68]:
chunker = pd.read_csv('data/io/big.csv', chunksize=1000)
chunker

In [69]:
chunker = pd.read_csv('data/io/big.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)  # instead .order()
tot[:10]

<ipython-input-69-8652bd39917e>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = Series([])


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing data out to text format

In [70]:
data = pd.read_csv('data/io/mis_val.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [71]:
data.to_csv('data/io/out.csv')
!cat data/io/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [72]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [73]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [74]:
# Supres row and column names
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [75]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [76]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('data/io/tseries.csv')
!cat data/io/tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [77]:
pd.read_csv('data/io/tseries.csv', parse_dates=[0])

,Unnamed: 0,0
0,2000-01-01,0
1,2000-01-02,1
2,2000-01-03,2
3,2000-01-04,3
4,2000-01-05,4
5,2000-01-06,5
6,2000-01-07,6


### Manually working with delimited formats

In [78]:
!cat data/io/coma_sep.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [79]:
import csv
f = open('data/io/coma_sep.csv')

reader = csv.reader(f)

for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [80]:
lines = list(csv.reader(open('data/io/coma_sep.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [81]:
class MyDialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [82]:
with open('data/io/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=MyDialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [83]:
%cat data/io/mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON data

In [84]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [85]:
import json
result = json.loads(obj)
asjson = json.dumps(result)

In [86]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


## Binary data formats

In [87]:
frame = pd.read_csv('data/io/io_1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [88]:
frame.to_pickle('data/io/frame_pickle')
pd.read_pickle('data/io/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### Using HDF5 format (hierarchical data format)
##### Support compress on fly
##### PyTables и h5py

In [ ]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
resp

In [91]:
issue_labels = DataFrame(resp)
issue_labels

,0
0,"b'[{""id"":76811,""node_id"":""MDU6TGFiZWw3NjgxMQ==..."
1,"b'lor"":""e10c02"",""default"":false,""description"":..."
2,"b'os/pandas-dev/pandas/labels/Enhancement"",""na..."
3,"b'""node_id"":""MDU6TGFiZWwxMjc2ODE="",""url"":""http..."
4,"b'"":""FCE94F"",""default"":false,""description"":""In..."
5,b'tps://api.github.com/repos/pandas-dev/pandas...
6,"b'y building on various platforms""},{""id"":1346..."
7,"b'andas/labels/Docs"",""name"":""Docs"",""color"":""34..."
8,"b'A="",""url"":""https://api.github.com/repos/pand..."
9,"b'e,""description"":null},{""id"":233160,""node_id""..."
